# **Deep Learning Course**

## **Loss Functions and Multilayer Perceptrons (MLP)**

---

### **Student Information:**

- **Name:** *ahmadreza majlesara*
- **Student Number:** *400101861*

---

### **Assignment Overview**

In this notebook, we will explore various loss functions used in neural networks, with a specific focus on their role in training **Multilayer Perceptrons (MLPs)**. By the end of this notebook, you will have a deeper understanding of:
- Types of loss functions
- How loss functions affect the training process
- The relationship between loss functions and model optimization in MLPs

---

### **Table of Contents**

1. Introduction to Loss Functions
2. Types of Loss Functions
3. Multilayer Perceptrons (MLP)
4. Implementing Loss Functions in MLP
5. Conclusion

---



# 1.Introduction to Loss Functions 

In deep learning, **loss functions** play a crucial role in training models by quantifying the difference between the predicted outputs and the actual targets. Selecting the appropriate loss function is essential for the success of your model. In this assay, we will explore various loss functions available in PyTorch, understand their theoretical backgrounds, and provide you with a scaffolded class to experiment with these loss functions.

Before begining, let's train a simle MLP model using the **L1Loss** function. We'll return to this model later to experiment with different loss functions. We'll start by importing the necessary libraries and defining the model architecture.

First things first, let's talk about **L1Loss**.

### 1. L1Loss (`torch.nn.L1Loss`)
- **Description:** Also known as Mean Absolute Error (MAE), L1Loss computes the average absolute difference between the predicted values and the target values.
- **Use Case:** Suitable for regression tasks where robustness to outliers is desired.

Here is the mathematical formulation of L1Loss:
\begin{equation}
\text{L1Loss} = \frac{1}{n} \sum_{i=1}^{n} |y_{\text{pred}_i} - y_{\text{true}_i}|
\end{equation}

Let's implement a simple MLP model using the L1Loss function.

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
from torch.utils.data import TensorDataset, DataLoader
from sklearn.model_selection import train_test_split
from torch.optim import Adam
# Don't be courious about Adam, it's just a fancy name for a fancy optimization algorithm

Here, we'll define a class called `SimpleMLP` that inherits from `nn.Module`. This class can have multiple layers, and we'll use the `nn.Sequential` module to define the layers of the model. The model will have the following architecture:

In [2]:
class SimpleMLP(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, num_hidden_layers=1, last_layer_activation_fn=nn.ReLU):
        super(SimpleMLP, self).__init__()
        layers = []
        layers.append(nn.Linear(input_dim, hidden_dim))
        layers.append(nn.ReLU())
        for _ in range(num_hidden_layers - 1):
            layers.append(nn.Linear(hidden_dim, hidden_dim))
            layers.append(nn.ReLU())
        layers.append(nn.Linear(hidden_dim, output_dim))
        if last_layer_activation_fn is not None:
            layers.append(last_layer_activation_fn())
        self.model = nn.Sequential(*layers)

    def forward(self, x):
        return self.model(x)

Now, let's define a class called `SimpleMLP_Loss` that has the following architecture:

In [3]:
from tqdm import tqdm

class SimpleMLPTrainer:
    def __init__(self, model, criterion, optimizer):
        self.model = model
        self.criterion = criterion
        self.optimizer = optimizer

    def train(self, train_loader, num_epochs):
        for epoch in range(num_epochs):
            self.model.train()
            running_loss = 0.0
            correct = 0
            total = 0
            for inputs, targets in tqdm(train_loader):
                self.optimizer.zero_grad()
                outputs = self.model(inputs)
                loss = self.criterion(outputs, targets)
                loss.backward()
                self.optimizer.step()
                running_loss += loss.item()
                predicted = (outputs > 0.5).float()
                total += targets.size(0)
                correct += (predicted == targets).sum().item()
            
            accuracy = 100 * correct / total
            print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader):.4f}, Accuracy: {accuracy:.2f}%")


    def evaluate(self, val_loader):
        self.model.eval()
        correct = 0
        total = 0
        with torch.no_grad():
            for inputs, targets in val_loader:
                outputs = self.model(inputs)
                predicted = (outputs > 0.5).float()
                total += targets.size(0)
                correct += (predicted == targets).sum().item()

        accuracy = 100 * correct / total
        print(f"Validation Accuracy: {accuracy:.2f}%")
        return accuracy


Next, lets test our model using the L1Loss function. You'll use <span style="color:red">*Titanic Dataset*</span> to train the model.


In [4]:
# Load dataset
train_url = "https://raw.githubusercontent.com/datasciencedojo/datasets/master/titanic.csv"
data = pd.read_csv(train_url)

# Preprocessing (simple example)
data = data[['Pclass', 'Sex', 'Age', 'Fare', 'Survived']].dropna()
data['Sex'] = data['Sex'].map({'male': 0, 'female': 1})

# TODO: Convert the data to PyTorch tensors and create a DataLoader

# Separate features and labels
X = data[['Pclass', 'Sex', 'Age', 'Fare']].values
y = data['Survived'].values

# Convert to PyTorch tensors
X_tensor = torch.tensor(X, dtype=torch.float32)
y_tensor = torch.tensor(y, dtype=torch.float32).unsqueeze(1)

# TODO: Split the data into training and validation sets

X_train, X_val, y_train, y_val = train_test_split(X_tensor, y_tensor, test_size=0.2, random_state=42)

train_dataset = TensorDataset(X_train, y_train)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

val_dataset = TensorDataset(X_val, y_val)
val_loader = DataLoader(val_dataset, batch_size=32)
# TODO: Define the model, criterion, and optimizer

input_dim = X_train.shape[1]
hidden_dim = 16
output_dim = 1


<div style="text-align: center;"> <span style="color:red; font-size: 26px; font-weight: bold;">Let's train!</span> </div>

In [5]:
from torch.nn import L1Loss
model = SimpleMLP(input_dim, hidden_dim, output_dim)
criterion = nn.L1Loss()
optimizer = Adam(model.parameters(), lr=0.001)
# TODO: Train the model
num_epochs = 50

mlptrainer = SimpleMLPTrainer(model, criterion, optimizer)
mlptrainer.train(train_loader, num_epochs)

mlptrainer.evaluate(val_loader)

100%|██████████| 18/18 [00:00<00:00, 216.91it/s]


Epoch [1/50], Loss: 0.4101, Accuracy: 59.02%


100%|██████████| 18/18 [00:00<00:00, 327.28it/s]


Epoch [2/50], Loss: 0.4098, Accuracy: 59.02%


100%|██████████| 18/18 [00:00<00:00, 321.73it/s]


...

100%|██████████| 18/18 [00:00<00:00, 429.18it/s]


Epoch [49/50], Loss: 0.4101, Accuracy: 59.02%


100%|██████████| 18/18 [00:00<00:00, 396.77it/s]

Epoch [50/50], Loss: 0.4091, Accuracy: 59.02%
Validation Accuracy: 60.84%


60.83916083916084

---
# 2. Types of Loss Functions

PyTorch offers a variety of built-in loss functions tailored for different types of problems, such as regression, classification, and more. Below, we discuss several commonly used loss functions, their theoretical foundations, and typical use cases.

### 2. MSELoss (`torch.nn.MSELoss`)
- **Description:** Mean Squared Error (MSE) calculates the average of the squares of the differences between predicted and target values.
- **Use Case:** Commonly used in regression problems where larger errors are significantly penalized.

Here is boring math stuff for MSE:
\begin{equation}
\text{MSE} = \frac{1}{n} \sum_{i=1}^{n} (y_{i} - \hat{y}_{i})^{2}
\end{equation}

<span style="color:red; font-size: 18px; font-weight: bold;">Warning:</span> Don't forget to reinitialize the model before experimenting with different loss functions.

In [6]:
from torch.nn import MSELoss

# Define the model, criterion, and optimizer
input_dim = X_train.shape[1]
hidden_dim = 16
output_dim = 1

model = SimpleMLP(input_dim, hidden_dim, output_dim, num_hidden_layers=2)
criterion = MSELoss()
optimizer = Adam(model.parameters(), lr=0.001)

# Train the model
num_epochs = 50
mlptrainer = SimpleMLPTrainer(model, criterion, optimizer)
mlptrainer.train(train_loader, num_epochs)

# Evaluate the model

mlptrainer.evaluate(val_loader)

100%|██████████| 18/18 [00:00<00:00, 257.15it/s]


Epoch [1/50], Loss: 0.4095, Accuracy: 59.02%


100%|██████████| 18/18 [00:00<00:00, 285.73it/s]


Epoch [2/50], Loss: 0.4111, Accuracy: 59.02%


100%|██████████| 18/18 [00:00<00:00, 300.07it/s]


...

100%|██████████| 18/18 [00:00<00:00, 359.97it/s]


Epoch [49/50], Loss: 0.4095, Accuracy: 59.02%


100%|██████████| 18/18 [00:00<00:00, 346.14it/s]

Epoch [50/50], Loss: 0.4108, Accuracy: 59.02%
Validation Accuracy: 60.84%


60.83916083916084

### 3. NLLLoss (`torch.nn.NLLLoss`)
- **Description:** Negative Log-Likelihood Loss measures the likelihood of the target class under the predicted probability distribution.
- **Use Case:** Typically used in multi-class classification tasks, especially when combined with `log_softmax` activation.

Here is the mathematical formulation of NLLLoss:
\begin{equation}
\text{NLLLoss} = -\frac{1}{n} \sum_{i=1}^{n} \log(y_{i})
\end{equation}

I hope you note the logarithm in the formula. It's important! 

Why?

In this part, run your training with Relu at last layer. <span style="color:red; font-weight: bold;">Discuss </span> and explain the difference between the results of the two models. Find a proper solution to the problem.


In [ ]:
from torch.nn import NLLLoss

# Define the model, criterion, and optimizer
input_dim = X_train.shape[1]
hidden_dim = 16
output_dim = 2  

model = SimpleMLP(input_dim, hidden_dim, output_dim, num_hidden_layers=2 , last_layer_activation_fn=nn.ReLU)
criterion = NLLLoss()
optimizer = Adam(model.parameters(), lr=0.001)

# Train the model
num_epochs = 50

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    for inputs, targets in tqdm(train_loader):
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets.long().squeeze())
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        
        _, predicted = torch.max(outputs, 1)
        total += targets.size(0)
        correct += (predicted == targets.long().squeeze()).sum().item()
    
    accuracy = 100 * correct / total
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader):.4f}, Accuracy: {accuracy:.2f}%")

model.eval()
correct = 0
total = 0
with torch.no_grad():
    for inputs, targets in val_loader:
        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)
        total += targets.size(0)
        correct += (predicted == targets.long().squeeze()).sum().item()

accuracy = 100 * correct / total
print(f"Validation Accuracy: {accuracy:.2f}%")

100%|██████████| 18/18 [00:00<00:00, 204.56it/s]


Epoch [1/50], Loss: -2.3914, Accuracy: 40.98%


100%|██████████| 18/18 [00:00<00:00, 211.74it/s]


Epoch [2/50], Loss: -3.9506, Accuracy: 40.98%


100%|██████████| 18/18 [00:00<00:00, 243.26it/s]


...

Epoch [48/50], Loss: -16042.2786, Accuracy: 40.98%


100%|██████████| 18/18 [00:00<00:00, 130.44it/s]


Epoch [49/50], Loss: -16965.6214, Accuracy: 40.98%


100%|██████████| 18/18 [00:00<00:00, 327.27it/s]

Epoch [50/50], Loss: -18020.3580, Accuracy: 40.98%
Validation Accuracy: 39.16%


In [ ]:
from torch.nn import NLLLoss

# Define the model, criterion, and optimizer
input_dim = X_train.shape[1]
hidden_dim = 16
output_dim = 2  

model = SimpleMLP(input_dim, hidden_dim, output_dim, num_hidden_layers=2 , last_layer_activation_fn=nn.LogSoftmax)
criterion = NLLLoss()
optimizer = Adam(model.parameters(), lr=0.001)

# Train the model
num_epochs = 50

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    for inputs, targets in tqdm(train_loader):
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets.long().squeeze())
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        
        _, predicted = torch.max(outputs, 1)
        total += targets.size(0)
        correct += (predicted == targets.long().squeeze()).sum().item()
    
    accuracy = 100 * correct / total
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader):.4f}, Accuracy: {accuracy:.2f}%")

model.eval()
correct = 0
total = 0
with torch.no_grad():
    for inputs, targets in val_loader:
        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)
        total += targets.size(0)
        correct += (predicted == targets.long().squeeze()).sum().item()

accuracy = 100 * correct / total
print(f"Validation Accuracy: {accuracy:.2f}%")

100%|██████████| 18/18 [00:00<00:00, 131.39it/s]


Epoch [1/50], Loss: 1.3361, Accuracy: 32.05%


100%|██████████| 18/18 [00:00<00:00, 160.72it/s]


Epoch [2/50], Loss: 0.7410, Accuracy: 50.09%


100%|██████████| 18/18 [00:00<00:00, 305.09it/s]


...

Epoch [48/50], Loss: 0.4761, Accuracy: 78.46%


100%|██████████| 18/18 [00:00<00:00, 300.01it/s]


Epoch [49/50], Loss: 0.4724, Accuracy: 78.28%


100%|██████████| 18/18 [00:00<00:00, 310.35it/s]

Epoch [50/50], Loss: 0.4749, Accuracy: 78.11%
Validation Accuracy: 73.43%


Your reason for your choice:

<div>

Since the `log_softmax` activation function is made to operate well with `NLLLoss`, which requires inputs to be log-probabilities, I utilized it in the last layer. For efficient loss computation, the `log_softmax` function makes sure the outputs are appropriately normalized as log-probabilities. ReLU, on the other hand, does not yield probabilistic outputs, which would result in inaccurate loss computations and impede the model's ability to learn. The model trains more efficiently, increasing accuracy and guaranteeing consistent training dynamics, when the activation function is matched to the criteria of the loss function.

</div>


### 4. CrossEntropyLoss (`torch.nn.CrossEntropyLoss`)
- **Description:** Combines `LogSoftmax` and `NLLLoss` in one single class. It computes the cross-entropy loss between the target and the output logits.
- **Use Case:** Widely used for multi-class classification problems.

The mathematical formulation of CrossEntropyLoss is as follows:
\begin{equation}
  \text{CrossEntropy}(y, \hat{y}) = - \sum_{i=1}^{C} y_i \log\left(\frac{e^{\hat{y}_i}}{\sum_{j=1}^{C} e^{\hat{y}_j}}\right)
\end{equation}
  where:
  - $ C $ is the number of classes,
  - $ y_i $ is a one-hot encoded target vector (or a scalar class label),
  - $ \hat{y}_i $ represents the logits (unnormalized model outputs) for each class.
  
  In practice, `torch.nn.CrossEntropyLoss` expects raw logits as input and internally applies the softmax function to convert the logits into probabilities, followed by the negative log-likelihood computation.

- **Background:** Cross-entropy measures the difference between the true distribution $ y $ and the predicted distribution $ \hat{y} $. The function minimizes the negative log-probability assigned to the correct class, effectively penalizing predictions that deviate from the true class, making it a standard choice for classification tasks in deep learning.

Now, let's implement a class called `SimpleMLP_Loss` that has the following architecture:


In [ ]:
from torch.nn import CrossEntropyLoss

# Define the model, criterion, and optimizer
input_dim = X_train.shape[1]
hidden_dim = 16
output_dim = 2  # For binary classification with CrossEntropyLoss, we need two output units

model = SimpleMLP(input_dim, hidden_dim, output_dim, num_hidden_layers=2)
criterion = CrossEntropyLoss()
optimizer = Adam(model.parameters(), lr=0.001)

# Train the model
num_epochs = 50

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    for inputs, targets in tqdm(train_loader):
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets.long().squeeze())
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        
        _, predicted = torch.max(outputs, 1)
        total += targets.size(0)
        correct += (predicted == targets.long().squeeze()).sum().item()
    
    accuracy = 100 * correct / total
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader):.4f}, Accuracy: {accuracy:.2f}%")

# Evaluate the model
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for inputs, targets in val_loader:
        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)
        total += targets.size(0)
        correct += (predicted == targets.long().squeeze()).sum().item()

accuracy = 100 * correct / total
print(f"Validation Accuracy: {accuracy:.2f}%")

100%|██████████| 18/18 [00:00<00:00, 142.87it/s]


Epoch [1/50], Loss: 0.7193, Accuracy: 53.06%


100%|██████████| 18/18 [00:00<00:00, 171.45it/s]


Epoch [2/50], Loss: 0.6709, Accuracy: 67.43%


100%|██████████| 18/18 [00:00<00:00, 299.99it/s]


...

Epoch [48/50], Loss: 0.5374, Accuracy: 73.20%


100%|██████████| 18/18 [00:00<00:00, 342.09it/s]


Epoch [49/50], Loss: 0.5516, Accuracy: 71.63%


100%|██████████| 18/18 [00:00<00:00, 315.77it/s]

Epoch [50/50], Loss: 0.5325, Accuracy: 74.08%
Validation Accuracy: 66.43%



### 5. KLDivLoss (`torch.nn.KLDivLoss`)
- **Description:** Kullback-Leibler Divergence Loss measures how one probability distribution diverges from a second, reference distribution. Unlike other loss functions that focus on classification, KL divergence specifically compares the relative entropy between two distributions. It quantifies the information loss when using the predicted distribution to approximate the true distribution. 

- **Mathematical Function:**
\begin{equation}
  \text{KL}(P \parallel Q) = \sum_{i=1}^{C} P(i) \left( \log P(i) - \log Q(i) \right)
\end{equation}
  where:
  - $ P $ is the target (true) probability distribution,
  - $ Q $ is the predicted distribution (often the output of `log_softmax`),
  - $ C $ is the number of classes.

  KL divergence is always non-negative, and it equals zero if the two distributions are identical. The loss function expects the model's output to be in the form of log-probabilities (using `log_softmax`) and compares this against a target probability distribution, which is typically a normalized distribution (using softmax).

- **Use Case:** KLDivLoss is frequently used in:
  - **Variational Autoencoders (VAEs):** In VAEs, KL divergence is used to measure how much the learned latent space distribution deviates from a prior distribution (often Gaussian).
  - **Knowledge Distillation:** In teacher-student models, KL divergence is used to transfer the "soft" knowledge from a teacher model to a student model by comparing their output probability distributions.
  - **Reinforcement Learning:** It can be used to update policies while minimizing the divergence from a previous policy.

- **Background:** Kullback-Leibler divergence, a core concept in information theory, measures the inefficiency of assuming the predicted distribution $ Q $ when the true distribution is $ P $. It is asymmetric, meaning that $ KL(P \parallel Q) \neq KL(Q \parallel P) $, so the direction of the comparison matters.

Again, in this part, run your training with Relu at last layer. <span style="color:red; font-weight: bold;">Discuss </span> and explain the difference between the results of the two models. Find a proper solution to the problem.


In [ ]:
from torch.nn import KLDivLoss


input_dim = X_train.shape[1]
hidden_dim = 16
output_dim = 2  

model = SimpleMLP(input_dim, hidden_dim, output_dim, num_hidden_layers=2 , last_layer_activation_fn=nn.ReLU)
criterion = KLDivLoss(reduction='batchmean')
optimizer = Adam(model.parameters(), lr=0.001)

# Train the model
num_epochs = 50

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    for inputs, targets in tqdm(train_loader):
        optimizer.zero_grad()
        outputs = model(inputs)
        log_outputs = torch.log_softmax(outputs, dim=1)  
        targets_one_hot = torch.nn.functional.one_hot(targets.long().squeeze(), num_classes=output_dim).float()
        loss = criterion(log_outputs, targets_one_hot)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        
        _, predicted = torch.max(outputs, 1)
        total += targets.size(0)
        correct += (predicted == targets.long().squeeze()).sum().item()
    
    accuracy = 100 * correct / total
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader):.4f}, Accuracy: {accuracy:.2f}%")

# Evaluate the model
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for inputs, targets in val_loader:
        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)
        total += targets.size(0)
        correct += (predicted == targets.long().squeeze()).sum().item()

accuracy = 100 * correct / total
print(f"Validation Accuracy: {accuracy:.2f}%")

100%|██████████| 18/18 [00:00<00:00, 189.51it/s]


Epoch [1/50], Loss: 0.7178, Accuracy: 55.52%


100%|██████████| 18/18 [00:00<00:00, 197.81it/s]


Epoch [2/50], Loss: 0.6931, Accuracy: 63.57%


100%|██████████| 18/18 [00:00<00:00, 216.90it/s]


...

Epoch [48/50], Loss: 0.4953, Accuracy: 75.48%


100%|██████████| 18/18 [00:00<00:00, 285.71it/s]


Epoch [49/50], Loss: 0.4909, Accuracy: 78.11%


100%|██████████| 18/18 [00:00<00:00, 295.11it/s]

Epoch [50/50], Loss: 0.5057, Accuracy: 74.96%
Validation Accuracy: 65.03%


In [ ]:
from torch.nn import KLDivLoss


input_dim = X_train.shape[1]
hidden_dim = 16
output_dim = 2  

model = SimpleMLP(input_dim, hidden_dim, output_dim, num_hidden_layers=2 , last_layer_activation_fn=nn.LogSoftmax)
criterion = KLDivLoss(reduction='batchmean')
optimizer = Adam(model.parameters(), lr=0.001)

# Train the model
num_epochs = 50

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    for inputs, targets in tqdm(train_loader):
        optimizer.zero_grad()
        outputs = model(inputs)
        log_outputs = torch.log_softmax(outputs, dim=1)  
        targets_one_hot = torch.nn.functional.one_hot(targets.long().squeeze(), num_classes=output_dim).float()
        loss = criterion(log_outputs, targets_one_hot)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        
        _, predicted = torch.max(outputs, 1)
        total += targets.size(0)
        correct += (predicted == targets.long().squeeze()).sum().item()
    
    accuracy = 100 * correct / total
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader):.4f}, Accuracy: {accuracy:.2f}%")

# Evaluate the model
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for inputs, targets in val_loader:
        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)
        total += targets.size(0)
        correct += (predicted == targets.long().squeeze()).sum().item()

accuracy = 100 * correct / total
print(f"Validation Accuracy: {accuracy:.2f}%")

100%|██████████| 18/18 [00:00<00:00, 75.32it/s]


Epoch [1/50], Loss: 2.1028, Accuracy: 59.02%


100%|██████████| 18/18 [00:00<00:00, 230.78it/s]


Epoch [2/50], Loss: 0.7367, Accuracy: 57.97%


100%|██████████| 18/18 [00:00<00:00, 243.24it/s]


...

Epoch [48/50], Loss: 0.5370, Accuracy: 73.03%


100%|██████████| 18/18 [00:00<00:00, 285.69it/s]


Epoch [49/50], Loss: 0.5430, Accuracy: 71.63%


100%|██████████| 18/18 [00:00<00:00, 227.86it/s]

Epoch [50/50], Loss: 0.5357, Accuracy: 73.20%
Validation Accuracy: 65.73%


Your reason for your choice:

<div>

Because the `log_softmax` activation function is especially designed to work with `KLDivLoss`, which requires that the model's outputs be log-probabilities, I utilized it in the last layer. Accurate measurement of the difference between the true and predicted distributions is made possible by the `log_softmax` function, which ensures that the outputs form a proper log-probability distribution. ReLU, on the other hand, does not generate outputs that can be interpreted probabilistically, which leads to poor model performance and inefficient loss computation. By matching the activation function to the loss function's specifications, the model can train more efficiently, lowering validation loss and increasing accuracy. Additionally, this alignment guarantees well-behaved optimization and meaningful gradients.
</div>

### 6. CosineEmbeddingLoss (`torch.nn.CosineEmbeddingLoss`)
- **Description:** Measures the cosine similarity between two input tensors, `x1` and `x2`, and computes the loss based on a label `y` that indicates whether the tensors should be similar (`y = 1`) or dissimilar (`y = -1`). Cosine similarity focuses on the angle between vectors, disregarding their magnitude.

- **Mathematical Function:** 
\begin{equation}
  \text{CosineEmbeddingLoss}(x1, x2, y) = 
  \begin{cases} 
  1 - \cos(x_1, x_2), & \text{if } y = 1 \\
  \max(0, \cos(x_1, x_2) - \text{margin}), & \text{if } y = -1
  \end{cases}
\end{equation}
  where $ \cos(x_1, x_2) $ is the cosine similarity between the two vectors, and `margin` is a threshold that determines how dissimilar the vectors should be.

- **Use Case:** Commonly used in tasks like face verification, image similarity, and other scenarios where the relative orientation of vectors (angle) is more important than their length, such as in embeddings and metric learning.

- **Background:** Cosine similarity compares the directional alignment of vectors, making it ideal for high-dimensional data where the magnitude may not be as informative. This loss is particularly useful when training models to learn meaningful embeddings that capture semantic similarity.

You'll become more fimiliar with this loss function in future.

---

# Regularization in Machine Learning

## Introduction

Regularization is a fundamental technique in machine learning that helps prevent overfitting by adding a penalty to the loss function. This penalty discourages the model from becoming too complex, ensuring better generalization to unseen data. In this notebook, you will explore the concepts of regularization, understand different types of regularization techniques, and apply them using Python's popular libraries.

## What is Regularization?

Regularization involves adding a regularization term to the loss function used to train machine learning models. This term imposes a constraint on the model's coefficients, effectively reducing their magnitude. By doing so, regularization helps in:

- **Preventing Overfitting:** Ensures the model does not become too tailored to the training data.
- **Improving Generalization:** Enhances the model's performance on new, unseen data.
- **Feature Selection:** Especially in L1 regularization, it can drive some coefficients to zero, effectively selecting important features.

## Types of Regularization

There are several types of regularization techniques, each imposing different constraints on the model's parameters:

### 1. L1 Regularization (Lasso)

L1 regularization adds the absolute value of the magnitude of coefficients as a penalty term to the loss function. It can lead to sparse models where some feature coefficients are exactly zero.

### 2. L2 Regularization (Ridge)

L2 regularization adds the squared magnitude of coefficients as a penalty term to the loss function. It tends to shrink the coefficients evenly but does not set them to zero.

### 3. Elastic Net

Elastic Net combines both L1 and L2 regularization penalties. It balances the benefits of both Lasso and Ridge methods, allowing for feature selection and coefficient shrinkage.

## Homework Time!
Import Iris dataset from sklearn.datasets and apply ridge regression with different alpha values. Then, create a gif that shows the changes of the classification boundary with respect to alpha values.

Import the libs that you need and start coding!

In [15]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import load_iris
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from PIL import Image
from io import BytesIO
import imageio
import warnings


# Suppress warnings for cleaner output
warnings.filterwarnings("ignore")

Load the Iris dataset and select Setosa and Versicolor classes

In [16]:
# 1. Load and Prepare the Iris Dataset
iris = load_iris()
X = iris.data
y = iris.target

# Select only two classes for binary classification (Setosa and Versicolor)
binary_class_indices = np.where((y == 0) | (y == 1))
X = X[binary_class_indices]
y = y[binary_class_indices]

# Select two features for 2D visualization (Sepal Length and Petal Length)
X = X[:, [0, 2]]

# Split into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

Define Function to Plot Decision Boundary

In [ ]:
def plot_decision_boundary(model, X, y, alpha):
    x_min, x_max = X[:, 0].min() - 1, X[:, 0].max() + 1
    y_min, y_max = X[:, 1].min() - 1, X[:, 1].max() + 1
    xx, yy = np.meshgrid(np.arange(x_min, x_max, 0.01),
                         np.arange(y_min, y_max, 0.01))

    grid = np.c_[xx.ravel(), yy.ravel()]
    with torch.no_grad():
        grid_tensor = torch.tensor(grid, dtype=torch.float32)
        Z = model(grid_tensor).numpy()
        Z = np.argmax(Z, axis=1)
    Z = Z.reshape(xx.shape)

    fig, ax = plt.subplots(figsize=(6, 5))

    ax.contourf(xx, yy, Z, alpha=0.3, levels=[-0.1, 0.1, 1.1], colors=['blue', 'red'])

    scatter = ax.scatter(
        X[:, 0], X[:, 1], c=y, cmap='bwr', edgecolor='k', s=50
    )

    ax.set_title(f'MLP Decision Boundary (alpha={alpha})')
    ax.set_xlabel('Sepal Length (standardized)')
    ax.set_ylabel('Petal Length (standardized)')

    ax.set_xticks([])
    ax.set_yticks([])

    plt.tight_layout()

    buf = BytesIO()
    plt.savefig(buf, format='png')
    plt.close(fig)
    buf.seek(0)
    return Image.open(buf)

Train MLP with Varying Alpha Values and Collect Images

In [ ]:
def create_decision_boundary_gif(alpha_values, X_train, y_train, n_neurons):

    # List to store images
    images = []

    for idx, alpha in enumerate(alpha_values):
        print(f"Processing alpha={alpha:.4f} ({idx + 1}/{len(alpha_values)})")

        # Create and train the MLP
        mlp = SimpleMLP(input_dim=2, hidden_dim=n_neurons, output_dim=2)
        criterion = nn.CrossEntropyLoss()
        optimizer = Adam(mlp.parameters(), lr=0.01, weight_decay=alpha)

        # Convert data to tensors
        X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
        y_train_tensor = torch.tensor(y_train, dtype=torch.long)

        # Training loop
        mlp.train()
        for epoch in range(1000):  
            optimizer.zero_grad()
            outputs = mlp(X_train_tensor)
            loss = criterion(outputs, y_train_tensor)
            loss.backward()
            optimizer.step()

        # Plot decision boundary and get the image
        img = plot_decision_boundary(mlp, X_train, y_train, alpha)
        images.append(img)

    # Save the images as a GIF
    gif_filename = 'mlp_classification_boundaries.gif'
    images[0].save(
        gif_filename,
        save_all=True,
        append_images=images[1:],
        duration=500,
        loop=0
    )

    print(f"GIF saved as '{gif_filename}'")

    # return the gif
    return gif_filename

## RUN

In [ ]:
alpha_values = np.logspace(-4, 2, 20)

n_neurons = 10

gif_dir = create_decision_boundary_gif(alpha_values, X_train, y_train, n_neurons)

Processing alpha=0.0001 (1/20)
Processing alpha=0.0002 (2/20)
Processing alpha=0.0004 (3/20)
Processing alpha=0.0009 (4/20)
Processing alpha=0.0018 (5/20)
Processing alpha=0.0038 (6/20)
Processing alpha=0.0078 (7/20)
Processing alpha=0.0162 (8/20)
Processing alpha=0.0336 (9/20)
Processing alpha=0.0695 (10/20)
Processing alpha=0.1438 (11/20)
Processing alpha=0.2976 (12/20)
Processing alpha=0.6158 (13/20)
Processing alpha=1.2743 (14/20)
Processing alpha=2.6367 (15/20)
Processing alpha=5.4556 (16/20)
Processing alpha=11.2884 (17/20)
Processing alpha=23.3572 (18/20)
Processing alpha=48.3293 (19/20)
Processing alpha=100.0000 (20/20)
GIF saved as 'mlp_classification_boundaries.gif'


Your gif should look like this:

<div style="text-align: center;">

### **Multilayer Perceptron Classification Boundaries**

![Classification Boundaries](mlp_classification_boundaries_example.gif)

*Figure 1: Demonstration of classification boundaries created by a Multilayer Perceptron (MLP) model.*

</div>

